In [1]:
import findspark
findspark.init('/home/kakade/spark')
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession,SQLContext
spark = SparkSession.builder.appName("kakade").getOrCreate()

In [12]:
from pyspark.sql.types import StructField,StringType,FloatType,StructType
#Add header to data feature
feature_schema = [StructField("country", StringType(), True), StructField("sku_id", StringType(), True), \
                  StructField("title",StringType(), True), StructField("category_1",StringType(), True), \
                  StructField("category_2",StringType(), True), StructField("category_3",StringType(), True), \
                  StructField("description",StringType(), True), StructField("org_price", FloatType(), True), \
                  StructField("product_type",StringType(), True)]
# country sku_id title category_1 category_2 category_3 short_description price product_type 
feature_struc = StructType(fields=feature_schema)

In [13]:
train_feature = spark.read.csv("../Data/training/data_train.csv",escape='"',schema=feature_struc)

In [15]:
#most other category only contain <=5 products
#filter products out of the main categories
# train_feature.groupBy("category_1").count().orderBy("count",ascending=False).show(truncate=False)
main_category=train_feature.groupBy("category_1").count().filter("count>5")
train_feature = train_feature.join(main_category, col("train_feature.category_1") == col("main_category.category_1"), 'inner')

NameError: name 'col' is not defined

In [16]:
from bs4 import BeautifulSoup
from pyspark.sql.functions import udf
from pyspark.ml.feature import Tokenizer, RegexTokenizer,StopWordsRemover,CountVectorizer

In [17]:
def html_extract_func(col):
    soup=BeautifulSoup(col)
    return soup.get_text()
    return (1-col1/col2)**2
html_extract = udf(html_extract_func, StringType())
train_feature = train_feature.withColumn("html_extract",html_extract("description"))
# train_feature.select("html_extract").show(truncate=False)

In [18]:
train_feature.printSchema()

root
 |-- country: string (nullable = true)
 |-- sku_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- category_1: string (nullable = true)
 |-- category_2: string (nullable = true)
 |-- category_3: string (nullable = true)
 |-- description: string (nullable = true)
 |-- org_price: float (nullable = true)
 |-- product_type: string (nullable = true)
 |-- html_extract: string (nullable = true)



In [10]:
import pyspark.sql.functions as F
des_cat_test = train_feature.select("category_1","sku_id",F.regexp_replace(train_feature.html_extract, '(\d+)', ' ').alias('html_extract'))
regexTokenizer = RegexTokenizer(inputCol="html_extract", outputCol="description_token", pattern="\\W")
des_cat_test = regexTokenizer.transform(des_cat_test)
stop_words_remover = StopWordsRemover(inputCol="description_token", outputCol="stop_words_filtered")
des_cat_test = stop_words_remover.transform(des_cat_test)
des_cat_test.select("stop_words_filtered").show(truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|stop_words_filtered                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
+-------------------------------------------------------

In [13]:
##currency_exchange
#set all price to PHP
import pyspark.sql.functions as F
from forex_python.converter import CurrencyRates
cex = CurrencyRates()
S2P=cex.get_rate("SGD","PHP")
M2P=cex.get_rate("MYR","PHP")
train_feature=train_feature.withColumn("price", F.when(train_feature.country == "my", M2P*train_feature.org_price).when(train_feature.country == "sg", S2P*train_feature.org_price).otherwise(train_feature.org_price))
                                    

In [15]:
train_feature.select("price","org_price").show()

+------------------+---------+
|             price|org_price|
+------------------+---------+
|            597.31|     49.0|
|           1560.32|    128.0|
| 305.6032962799072|    25.07|
|1438.4199999999998|    118.0|
|1399.4120372009277|    114.8|
|31681.809999999998|   2599.0|
| 4741.787980957031|   388.99|
|126.77599534988403|     10.4|
|               0.0|      0.0|
|            304.75|     25.0|
|115.56119441986084|     9.48|
| 950.8199999999999|     78.0|
|189.55450232505797|    15.55|
|6082.8099999999995|    499.0|
|            1462.8|    120.0|
|              null|     null|
| 1031.273981399536|     84.6|
|             365.7|     30.0|
| 203.0853981399536|    16.66|
|2072.2999999999997|    170.0|
+------------------+---------+
only showing top 20 rows



In [20]:
des_cat_test.groupBy("category_1").count().orderBy("count",ascending=False).show(truncate=False)

+--------------------------------------------------------------+-----+
|category_1                                                    |count|
+--------------------------------------------------------------+-----+
|Mobiles & Tablets                                             |7273 |
|Home & Living                                                 |6042 |
|Fashion                                                       |5729 |
|Watches Sunglasses Jewellery                                  |4216 |
|Health & Beauty                                               |4040 |
|Computers & Laptops                                           |2882 |
|TV, Audio / Video, Gaming & Wearables                         |2505 |
|Cameras                                                       |1950 |
|Home Appliances                                               |1583 |
| 8GB                                                          |5    |
| 4GB                                                          |5    |
|A1466

In [32]:
main_category=des_cat_test.groupBy("category_1").count().filter("count>5")
cat_train_test = des_cat_test.join(main_category,des_cat_test.category_1=main_category.category_1, 'inner')

SyntaxError: keyword can't be an expression (<ipython-input-32-8532e3f6520c>, line 2)

In [5]:
train_feature.select("sku_id").describe().show()

+-------+------------------+
|summary|            sku_id|
+-------+------------------+
|  count|             36283|
|   mean|              null|
| stddev|              null|
|    min|1A127ELAA3U0JKANPH|
|    max|  ZY795HLCZ3O0ANMY|
+-------+------------------+



In [2]:
train_data=spark.read.csv("../Clean_Data_Frame.csv",escape='"',header=True)

In [3]:
test_sample=train_data.filter(train_data["category_1"]=="Fashion").drop("category_1","org_price","clarity","country")
test_sample=test_sample.na.fill({"description":""})

In [4]:
from pyspark.sql.functions import udf
from pyspark.ml.feature import Tokenizer, RegexTokenizer,StopWordsRemover,CountVectorizer
from pyspark.sql.types import StructField,StringType,FloatType,StructType,IntegerType
from pyspark.sql.functions import regexp_replace

def useful_token_func(col1,col2):
    if len(col1)>col2:
        return 0
    else:
        return col2-len(col1)
useful_token = udf(useful_token_func, IntegerType())

test_1 = test_sample.withColumn("title_no_sc1", regexp_replace(test_sample["title"], '[-/()]+', ' '))
test_1 = test_1.withColumn("title_no_sc", regexp_replace(test_1["title_no_sc1"], '''['.]+''', ''))

tokenizer = Tokenizer(inputCol="title_no_sc", outputCol="words")
regexTokenizer = RegexTokenizer(inputCol="title_no_sc", outputCol="regex_words", pattern="\\W")
stop_words_remover = StopWordsRemover(inputCol="regex_words", outputCol="no_stopwords")

countTokens = udf(lambda words: len(words), IntegerType())

test_1 = tokenizer.transform(test_1)
test_1=test_1.withColumn("tokens", countTokens(test_1["words"]))

test_1 = regexTokenizer.transform(test_1)
test_1=test_1.withColumn("regex_tokens", useful_token(test_1["regex_words"],test_1["tokens"]))

test_1 = stop_words_remover.transform(test_1)
test_1=test_1.withColumn("sw_tokens", useful_token(test_1["no_stopwords"],test_1["tokens"]))
test_1=test_1.withColumn("nsw_tokens", countTokens(test_1["no_stopwords"]))

In [5]:
test_1=test_1.drop("regex/org","sw/org","title","title_no_sc1")

In [6]:
test_2 = test_1.withColumn("desc_no_sc1", regexp_replace(test_sample["description"], '[-/()]+', ' '))
test_2 = test_2.withColumn("desc_no_sc", regexp_replace(test_2["desc_no_sc1"], '''['.]+''', ''))


regexTokenizer = RegexTokenizer(inputCol="desc_no_sc", outputCol="regex_des", pattern="\\W")
stop_words_remover = StopWordsRemover(inputCol="regex_des", outputCol="no_stopwords_des")

test_2 = regexTokenizer.transform(test_2)
test_2 = stop_words_remover.transform(test_2)
test_2=test_2.withColumn("des_tokens", countTokens(test_2["no_stopwords_des"]))
test_2=test_2.withColumn("r_tit_des",test_2["nsw_tokens"]/test_2["des_tokens"])

In [15]:
from pyspark.sql.types import ArrayType,StringType
uniqueExtract = udf(lambda words: list(set(words)),ArrayType(StringType()))
test_3=test_2.withColumn("unique_words",uniqueExtract(test_2["no_stopwords"]))
test_3=test_3.withColumn("count_repeat",countTokens(test_3["no_stopwords"])-test_3["nsw_tokens"])

In [25]:
from pyspark.ml.fpm import FPGrowth
fpGrowth = FPGrowth(itemsCol="unique_words", minSupport=0.1, minConfidence=0.5)
test_true=test_3.filter("concise==1")
fp_model = fpGrowth.fit(test_true)

# Display frequent itemsets.
fp_model.freqItemsets.show()

+---------+----+
|    items|freq|
+---------+----+
|  [shirt]| 285|
|  [women]| 379|
|  [dress]| 298|
|   [blue]| 312|
| [casual]| 287|
|   [intl]| 475|
|   [long]| 257|
| [sleeve]| 275|
|[fashion]| 319|
|   [mens]| 287|
|  [black]| 542|
+---------+----+



In [26]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer


hashingTF = HashingTF(inputCol="nsw_words", outputCol="rawFeatures", numFeatures=20)
featurizedData = hashingTF.transform(wordsData)
# alternatively, CountVectorizer can also be used to get term frequency vectors

idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(test_3)
rescaledData = idfModel.transform(featurizedData)

rescaledData.select("label", "features").show()

+----------------+----+
|           items|freq|
+----------------+----+
|       [leather]| 443|
|        [sleeve]| 391|
|          [intl]| 784|
|   [intl, women]| 348|
|          [long]| 458|
|         [dress]| 374|
|         [black]| 627|
|          [mens]| 384|
|           [new]| 452|
|          [blue]| 326|
|       [fashion]| 805|
|[fashion, women]| 376|
|           [men]| 422|
|          [sexy]| 433|
|        [womens]| 388|
|        [casual]| 634|
|           [bag]| 458|
|         [shoes]| 335|
|         [women]|1215|
+----------------+----+



In [32]:
test_1 = test_sample.withColumn("title_no_sc", regexp_replace(test_sample["title"], '[-]+', ' '))
countTokens = udf(lambda words: len(words), IntegerType())

test_1 = tokenizer.transform(test_sample)
test_1=test_1.withColumn("tokens", countTokens(test_1["words"]))

test_1 = regexTokenizer.transform(test_1)
test_1=test_1.withColumn("regex_tokens", countTokens(test_1["regex_words"]))

In [4]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(),
                          numFolds=2)

+-------+--------------------+--------------------+
|summary|               title|          category_1|
+-------+--------------------+--------------------+
|  count|               11417|               11417|
|   mean|              2016.0|                null|
| stddev|                 NaN|                null|
|    min|!!! -1.00 !!! Lar...|             Cameras|
|    max|���Always Smile�۝...|Watches Sunglasse...|
+-------+--------------------+--------------------+



In [5]:
train_data.filter("clarity==1").select("title","category_1").describe().show()

+-------+--------------------+--------------------+
|summary|               title|          category_1|
+-------+--------------------+--------------------+
|  count|               34228|               34228|
|   mean|                null|                null|
| stddev|                null|                null|
|    min|!!! -1.00 !!! Lar...|             Cameras|
|    max|���Always Smile�۝...|Watches Sunglasse...|
+-------+--------------------+--------------------+



In [6]:
train_data.filter("clarity==0").select("title","category_1").describe().show()

+-------+--------------------+--------------------+
|summary|               title|          category_1|
+-------+--------------------+--------------------+
|  count|                2055|                2055|
|   mean|              2016.0|                null|
| stddev|                 NaN|                null|
|    min|"ASTAR Lady's Str...|             Cameras|
|    max|yukufus MEGIR min...|Watches Sunglasse...|
+-------+--------------------+--------------------+

